# Converting the Modified Swiss Dwellings Dataset into TopologicPy Graphs
The Modified Swiss Dwellings is a machine learing-ready Floor Plan Dataset of Residential Building Complexes.
It was created at Delft University. The owner of the dataset is: Casper van Engelenburg.
The license for the dataset as well as this entire work and the dataset it creates is governed by:
the CC BY-SA 4.0 license (https://creativecommons.org/licenses/by-sa/4.0/).

This script converts the graphs found in the dataset into TopologicPy graphs. It preserves the semantic information by storing dictionary values at the graph, edge, and vertex levels. It also saves the polygons of the rooms in the dictionary of the vertices under the "geometry" key. However, it does not import structure, doors, or windows.
 

## Instructions

1. Pip install kagglehub: <code>pip install kagglehub</code>
2. Run the cell below <em>ONLY ONCE</em>
3. Make note of the printed <code>folder_path</code>

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
# Download latest version
folder_path = kagglehub.dataset_download("caspervanengelenburg/modified-swiss-dwellings")

print("Path to dataset files:", folder_path)
print("\nDone")

## Import the needed libraries

In [ ]:
# You don't need these two lines if you have pip installed topologicpy. Remove!
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Graph import Graph
from topologicpy.Helper import Helper

from pathlib import Path
import os
import warnings
from tqdm.auto import tqdm
import pandas as pd

# On my machine, I was getting a torch FutureWarning. I decided to silence it here.
warnings.filterwarnings(
    "ignore",
    message=r"You are using `torch\.load`.*weights_only=False",
    category=FutureWarning,
)

print("\nDone")

## Check Software Versions

In [ ]:
import sys
print("This workflow was designed and verified to work with python 3.11 and TopologicPy 0.8.60.\n")
py_version = sys.version.split()[0]
print("Python version:", py_version)

print("TopologicPy Version:", Helper.Version(check=False))
print(" ")
print("TopologicPy Version:", Helper.Version(check=True))
print("\nDONE")

## Set some constants

In [ ]:
ROOM_COLORS = ['#1f77b4',
                   '#e6550d',
                   '#fd8d3c',
                   '#fdae6b',
                   '#fdd0a2',
                   '#72246c',
                   '#5254a3',
                   '#6b6ecf',
                   '#2ca02c',
                   '#000000',
                   '#ffc000',
                   '#98df8a',
                   '#d62728']

ROOM_NUMBERS = {'Bedroom': 0,
                'Livingroom': 1,
                'Kitchen': 2,
                'Dining': 3,
                'Corridor': 4,
                'Stairs': 5,
                'Storeroom': 6,
                'Bathroom': 7,
                'Blacony': 8}

ROOM_NAMES = ['Bedroom',
              'Livingroom',
              'Kitchen',
              'Dining',
              'Corridor',
              'Stairs',
              'Storeroom',
              'Bathroom',
              'Balcony',
              'Structure',
              'Door',
              'Entrance Door',
              'Window']

ZONE_NUMBERS = {'Bedroom': 1,
                'Livingroom': 2,
                'Kitchen': 2,
                'Dining': 2,
                'Corridor': 2,
                'Stairs': 3,
                'Storeroom': 3,
                'Bathroom': 3,
                'Balcony': 4}

ZONE_NAMES = {'Bedroom': 'Zone1',
                'Livingroom': 'Zone2',
                'Kitchen': 'Zone2',
                'Dining': 'Zone2',
                'Corridor': 'Zone2',
                'Stairs': 'Zone3',
                'Storeroom': 'Zone3',
                'Bathroom': 'Zone3',
                'Balcony': 'Zone4'}

ZONE_COLORS = ['#1f77b4',
                 '#ff7f0e',
                 '#72246c',
                 '#2ca02c',
                 '#000000',
                 '#ffc000',
                 '#98df8a',
                 '#d62728']

META_KEYS = ['site_id',
             'building_id',
             'plan_id',
             'floor_id',
             'elevation',
             'height',
             'unit_usage']

print("\nDone")

## Set File Paths (Change according to your folder location)

In [ ]:
import os
folder_path = r"C:\Users\sarwj\.cache\kagglehub\datasets\caspervanengelenburg\modified-swiss-dwellings\versions\6"

# modified-swiss-dwellings-v2/
csv_file_path = os.path.join(folder_path, "mds_V2_5.372k.csv")
graph_out_folders = [os.path.join(folder_path, "modified-swiss-dwellings-v2","train", "graph_out"),
                     os.path.join(folder_path, "modified-swiss-dwellings-v2","test", "graph_out")]

# The desired output folder for your JSON files.
json_output_folder = r"C:\Users\sarwj\OneDrive - Cardiff University\Desktop\json_graphs"
print("\nDone")

## How many graphs do you want to export? Set max_n to None if you want all graphs to be exported (thousands).

In [ ]:
max_n = 5
print("\nDone")

In [ ]:

def filenames_without_ext(folder: str) -> list[str]:
    folder = Path(folder)
    return sorted(p.stem for p in folder.iterdir() if p.is_file())

import pickle

def load_pickle(filename):
    with open(filename, 'rb') as f:
        object = pickle.load(f)
        f.close()
    return object

# 1) Load CSV (the “main” dataframe shipped with the dataset)
df = pd.read_csv(csv_file_path)

for graph_out_folder in graph_out_folders:
    if "test" in graph_out_folder:
        label = "Test Dataset"
        meta_dict_py = {"ml_type": "test"}
    else:
        label = "Train Dataset"
        meta_dict_py = {"ml_type": "train"}
    file_names = filenames_without_ext(graph_out_folder)
    if not max_n:
        max_n = len(file_names)
    else:
        max_n = min(max_n, len(file_names))
    enumerator = tqdm(enumerate(file_names[:max_n]), desc=label, total=max_n, leave=False)
    for i, file_name in enumerator:
        floor_id = int(file_name)
        # get metadata from CSV file
        result = df.query('floor_id == @floor_id and entity_type == "area"')
        first_row_dict = result.iloc[0].to_dict() #All rows have the same meta key values
        
        for meta_key in META_KEYS:
            meta_dict_py[meta_key] = first_row_dict.get(meta_key, None)
        meta_dict = Dictionary.ByPythonDictionary(meta_dict_py)
        ng = load_pickle(os.path.join(graph_out_folder,file_name+".pickle"))
        g = Graph.ByNetworkXGraph(ng, coordsKey="centroid")
        vertices = Graph.Vertices(g)
        faces = []
        mesh_data = Graph.MeshData(g)
        edges = mesh_data['edges']
        edge_dicts = mesh_data['edgeDictionaries']
        vert_dicts = mesh_data['vertexDictionaries']
        verts = []
        new_v_dicts = []
        for v in vertices:
            d = Topology.Dictionary(v)
            geom = Dictionary.ValueAtKey(d, "geometry")
            node_type = Dictionary.ValueAtKey(d, "room_type")
            node_name = ROOM_NAMES[int(node_type)]
            node_color = ROOM_COLORS[int(node_type)]
            zone_type = ZONE_NUMBERS[node_name]
            zone_name = ZONE_NAMES[node_name]
            zone_color = ZONE_COLORS[zone_type]
            d = Dictionary.RemoveKey(d, "room_type")
            d = Dictionary.RemoveKey(d, "centroid")
            d = Dictionary.SetValuesAtKeys(d, ["type", "name", "color", "zone_type", "zone_name", "zone_color"], [node_type, node_name, node_color, zone_type, zone_name, zone_color])
            v = Topology.SetDictionary(v, d)
            geo_verts = [Vertex.ByCoordinates(coords) for coords in geom]
            geo_verts = Vertex.Fuse(geo_verts)
            geo_wire = Wire.ByVertices(geo_verts, close=True, silent=True)
            if Topology.IsInstance(geo_wire, "wire"):
                if  Wire.IsClosed(geo_wire):
                    geometry = [Vertex.Coordinates(v, outputType="xy", mantissa=6) for v in Topology.Vertices(geo_wire)]
                    d = Dictionary.SetValueAtKey(d, "geometry", geometry)
                    geo_face = Face.ByWire(geo_wire)
                    if Topology.IsInstance(geo_face, "face"):
                        faces.append(geo_face)
                        area = Face.Area(geo_face)
                        d = Dictionary.SetValuesAtKeys(d, ["area"], [area])

            verts.append(Vertex.Coordinates(v))
            new_v_dicts.append(Dictionary.PythonDictionary(d))

        new_g = Graph.ByMeshData(verts, edges, vertexDictionaries=new_v_dicts, edgeDictionaries=edge_dicts)
        new_g = Topology.SetDictionary(new_g, meta_dict)
        json_file_name = file_name+".json"
        json_path = os.path.join(json_output_folder,json_file_name)
        status = Graph.ExportToJSON(new_g, json_path, overwrite=True)
print("\nDone")

## Reload a sample graph (the last one in this case)

In [ ]:
graph = Graph.ByJSONPath(path = json_path)
vertices = Graph.Vertices(graph)
for v in vertices:
    d = Topology.Dictionary(v)
    d = Dictionary.SetValueAtKey(d, "vertexSize", 14)
    v = Topology.SetDictionary(v, d)
print("\nDone")

In [ ]:
Topology.Show(faces,
              graph,
              vertexSizeKey="vertexSize",
              vertexColorKey="color",
              showVertexLabel=True,
              vertexLabelKey="name",
              camera = [0,0,3],
              up = [0,1,0],
              backgroundColor="white")
print("\nDone")